In [ ]:
#IMPORTS
import pandas as pd
import logging
import torch
from collections import Counter
from ast import literal_eval
import torch
from urllib import request
import random
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
print(device)

cuda


In [ ]:
# # prepare logger
# logging.basicConfig(level=logging.INFO)

# transformers_logger = logging.getLogger("transformers")
# transformers_logger.setLevel(logging.WARNING)

# # check gpu
# cuda_available = torch.cuda.is_available()

# print('Cuda available? ',cuda_available)
# if cuda_available:
#   import tensorflow as tf
#   # Get the GPU device name.
#   device_name = tf.test.gpu_device_name()
#   # The device name should look like the following:
#   if device_name == '/device:GPU:0':
#       print('Found GPU at: {}'.format(device_name))
#   else:
#       raise SystemError('GPU device not found')

In [ ]:
module_url = f"https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py"
module_name = module_url.split('/')[-1]
print(f'Fetching {module_url}')
#with open("file_1.txt") as f1, open("file_2.txt") as f2
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  a = f.read()
  outf.write(a.decode('utf-8'))

# helper function to save predictions to an output file
def labels2file(p, outf_path):
	with open(outf_path,'w') as outf:
		for pi in p:
			outf.write(','.join([str(k) for k in pi])+'\n')

Fetching https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py


In [ ]:
from dont_patronize_me import DontPatronizeMe
dpm = DontPatronizeMe('.', '.')
dpm.load_task1()
dpm.load_task2(return_one_hot=True)

Map of label to numerical label:
{'Unbalanced_power_relations': 0, 'Shallow_solution': 1, 'Presupposition': 2, 'Authority_voice': 3, 'Metaphors': 4, 'Compassion': 5, 'The_poorer_the_merrier': 6}


In [ ]:
trids = pd.read_csv('train_semeval_parids-labels.csv')
teids = pd.read_csv('dev_semeval_parids-labels.csv')

trids.par_id = trids.par_id.astype(str)
teids.par_id = teids.par_id.astype(str)

data=dpm.train_task1_df
data

,par_id,art_id,keyword,country,text,label,orig_label
0,1,@@24942188,hopeless,ph,"We 're living in times of absolute insanity , ...",0,0
1,2,@@21968160,migrant,gh,"In Libya today , there are countless number of...",0,0
2,3,@@16584954,immigrant,ie,"""White House press secretary Sean Spicer said ...",0,0
3,4,@@7811231,disabled,nz,Council customers only signs would be displaye...,0,0
4,5,@@1494111,refugee,ca,""""""" Just like we received migrants fleeing El ...",0,0
...,...,...,...,...,...,...,...
10464,10465,@@14297363,women,lk,"""Sri Lankan norms and culture inhibit women fr...",0,1
10465,10466,@@70091353,vulnerable,ph,He added that the AFP will continue to bank on...,0,0
10466,10467,@@20282330,in-need,ng,""""""" She has one huge platform , and informatio...",1,3
10467,10468,@@16753236,hopeless,in,""""""" Anja Ringgren Loven I ca n't find a word t...",1,4


In [ ]:

rows = [] # will contain par_id, label and text
for idx in range(len(trids)):
  parid = trids.par_id[idx]
  #print(parid)
  # select row from original dataset to retrieve `text` and binary label
  keyword = data.loc[data.par_id == parid].keyword.values[0]
  text = data.loc[data.par_id == parid].text.values[0]
  label = data.loc[data.par_id == parid].label.values[0]
  orig_label = data.loc[data.par_id == parid].orig_label.values[0]
  item = {
      'par_id':parid,
      'community':keyword,
      'text':text,
      'label':label,
      'orig_label': orig_label
  }
  rows.append(item)

In [ ]:
trdf1 = pd.DataFrame(rows)
trdf1

,par_id,community,text,label
0,4341,poor-families,"The scheme saw an estimated 150,000 children f...",1
1,4136,homeless,Durban 's homeless communities reconciliation ...,1
2,10352,poor-families,The next immediate problem that cropped up was...,1
3,8279,vulnerable,Far more important than the implications for t...,1
4,1164,poor-families,To strengthen child-sensitive social protectio...,1
...,...,...,...,...
8370,8380,refugee,Rescue teams search for survivors on the rubbl...,0
8371,8381,hopeless,The launch of ' Happy Birthday ' took place la...,0
8372,8382,homeless,"The unrest has left at least 20,000 people dea...",0
8373,8383,hopeless,You have to see it from my perspective . I may...,0


In [ ]:
test_rows = [] # will contain par_id, label and text
for idx in range(len(teids)):
  parid = teids.par_id[idx]
  #print(parid)
  # select row from original dataset
  keyword = data.loc[data.par_id == parid].keyword.values[0]
  text = data.loc[data.par_id == parid].text.values[0]
  label = data.loc[data.par_id == parid].label.values[0]
  orig_label = data.loc[data.par_id == parid].orig_label.values[0]
  test_rows.append({
      'par_id':parid,
      'community':keyword,
      'text':text,
      'label':label,
      'orig_label': orig_label
  })
  #TODO: keyword?Country?length?

In [ ]:
tedf1 = pd.DataFrame(test_rows)
tedf1

,par_id,community,text,label
0,4046,hopeless,We also know that they can benefit by receivin...,1
1,1279,refugee,Pope Francis washed and kissed the feet of Mus...,1
2,8330,refugee,Many refugees do n't want to be resettled anyw...,1
3,4063,in-need,"""Budding chefs , like """" Fred """" , """" Winston ...",1
4,4089,homeless,"""In a 90-degree view of his constituency , one...",1
...,...,...,...,...
2089,10462,homeless,"The sad spectacle , which occurred on Saturday...",0
2090,10463,refugee,""""""" The Pakistani police came to our house and...",0
2091,10464,disabled,"""When Marie O'Donoghue went looking for a spec...",0
2092,10465,women,"""Sri Lankan norms and culture inhibit women fr...",0


In [ ]:
tedf1 = tedf1.sample(frac = 1) # shuffle data
# downsample negative instances
pcldf = trdf1[trdf1.label==1]
npos = len(pcldf)

training_set1 = pd.concat([pcldf,trdf1[trdf1.label==0][:npos*2]])
# training_set1 = pd.concat([training_set1, tedf1['community']], axis=1)
# tedf1.par_id
train_data = training_set1
test_data = tedf1
train_data_text_list = train_data.text.to_list()
train_data_label_list = train_data.label.to_list()
test_data_text_list = test_data.text.to_list()
test_data_label_list = test_data.label.to_list()
train_community = train_data.community.to_list()
test_community = test_data.community.to_list()
train_pcl = train_data.orig_label.to_list()
test_pcl = test_data.orig_label.to_list()
print(len(train_data_text_list))

In [ ]:
from sklearn.metrics import f1_score, f1_score
def calc_scores_and_print_miscalculations(true_lables, pred_labels, unseen_data, n=10):
    '''prints f1_score, f1 score and first N miscalculations'''
    calc_scores(true_lables, pred_labels)
    # Print predicted labels for unseen test data
    print("\nMisclassifications:")
    c = 0
    for text, true_label, pred_label in zip(unseen_data, true_lables, pred_labels):
        if true_label != pred_label:
            print(f"Pred: {pred_label} Ac: {true_label} Text: {text}")
            c += 1
            if c == n:
                break

def calc_scores(true_lables, pred_labels):
    # Evaluate the model
    f1_score = f1_score(true_lables, pred_labels)
    print("f1_score:", f1_score)

    f1 = f1_score(true_lables, pred_labels)
    print("F1 Score:", f1)
    return (f1_score, f1)

## Hyperparameter search - random search

In [ ]:
# @title Hyperparamter pipeline and final f1_score testing

# Objective function - takes in hyperparameters and returns a score we are trying
# to minimise

# Can use other loader if needed
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score
import numpy as np

def train_model(model, hyperparams: dict, train_split, test_split):
    ''' Train model and return f1_score'''
    # ------ EXAMPLE USE OF HYPERPARAMS IN MODEL LOOP --------
    # if optimizer == "SGD":
    #     optimizer = optim.SGD(model.parameters(), lr=hyperparams['LEARNING_RATE'], weight_decay=hyperparams['WEIGHT_DECAY'])
    ...
    return 0

def objective(hyperparams, iteration, train_split, test_split):
    # Update with own loader if needed
    model  = ...
    # all model parameters as key.value pairs in hyperparams
    f1_score = train_model(model, hyperparams, train_split, test_split)

    # update best hyperparameters
    return (f1_score, hyperparams, iteration)

# Domain
# Using logarithmic distrbution for learning rate creates a domain space where same
# number of values between 0.005 and 0.05 as 0.05 and 0.5.
param_grid = {
    "NUM_EPOCHS": [1,2],
    "SCHEDULER": [None, "Exponential", "Step"],
    "OPTIMIZER": ["Adam", "SGD"],
    "LEARNING_RATE" : list(np.logspace(np.log10(0.0001), np.log10(0.001), base = 10, num = 1000)),
    "BATCH_SIZE": [32,64,128],
    "GAMMA": list(np.linspace(0.05, 0.5, 5)),
    "WEIGHT_DECAY": list(np.logspace(np.log10(1e-7), np.log10(1e-5), base = 10, num = 1000)),
    "STEP_SIZE": list(range(5, 8))
}

import csv
import random

seed = random.randint(0, 10000000000)

# Create file and open connection
out_file = f"random_search_trials{seed}.csv"
of_connection = open(out_file, 'w')
writer = csv.writer(of_connection)

# Write column names
headers = ['score', 'hyperparameters', 'iteration']
writer.writerow(headers)
of_connection.close()

random.seed(seed)
print("seed", seed)
test_split, train_split = ..., ...
MAX_EVALS = 10
def random_search(param_grid, max_evals):
    results = [] #(score, hyperparams, iteration)

    # Keep searching until reach max evaluations
    for i in range(max_evals):

        # Choose random hyperparameters
        hyperparameters = {k: random.sample(v, 1)[0] for k, v in param_grid.items()}

        # Evaluate randomly selected hyperparameters
        print("Iteration ", i, " trying ", hyperparameters)
        res = objective(hyperparameters, i, test_split, train_split)

        results.append(res)

        # open connection (append option) and write results
        of_connection = open(out_file, 'a')
        writer = csv.writer(of_connection)
        writer.writerow(res)

        # make sure to close connection
        of_connection.close()
    # Sort with best score on top
    results.sort(reverse=True, key=lambda x : x[0])
    return results

random_results = random_search(param_grid, MAX_EVALS)

print('The best f1 score was {:.5f}'.format(random_results[0][0]))
print('\nThe best hyperparameters were:')

import pprint
best_search_params = random_results[0][1]
pprint.pprint(best_search_params)

FINAL_ACC = True

#BEST
# {'SCHEDULER': 'Step', 'OPTIMIZER': 'Adam', 'LEARNING_RATE': 0.0009908228099003798, 'BATCH_SIZE': 64}

if FINAL_ACC:
    # Create, train, and test model using best performing parameters
    hyperparams = best_search_params
    model = ...
    test_set_f1_score = train_model(model, hyperparams, train_split, test_split)
    print(f"The best model from random search scores {test_set_f1_score} on the test set")

    # save the model
    torch.save(model.state_dict(), 'model.pt')

In [ ]:
import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords
import random

# Download WordNet if not already downloaded
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

## Pre-processing/Augmentatoin and sampling helper methods

In [ ]:
!pip install googletrans
!pip install deep_translator

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 10.2 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.0.0-py3-none-any.whl size=15715 sha256=db112a170e0d9384b16e6c5e5ed89718668774813d3234aac5e896b212ba83cc
  Stored in directory: /root/.cache/pip/wheels/b3/81/ea/8b030407f8ebfc2f857814e086bb22ca2d4fea1a7be63652ab
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling ch

In [ ]:
# Synonm augmentation
stop_words = set(stopwords.words('english'))

def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name())
    return list(synonyms)

def synonym_replacement(text, n=3):
    words = text.split()
    augmented_texts = []
    for _ in range(n):
        new_words = words.copy()
        for i, word in enumerate(words):
            if word.lower() not in stop_words:  # Check if the word is not a stop word
                synonyms = get_synonyms(word)
                if synonyms:
                    synonym = random.choice(synonyms)
                    new_words[i] = synonym
        augmented_texts.append(' '.join(new_words))
    return augmented_texts

# Example usage:
original_text = "Somebody destroyed my only toy"
augmented_texts = synonym_replacement(original_text, n=3)  # Generate 3 augmented texts
for i, augmented_text in enumerate(augmented_texts, 1):
    print(f"Augmented Text {i}: {augmented_text}")


def replacement_text(text_list, label_list, transform_f=synonym_replacement):
    aug_text_list = []
    aug_label_list = []
    for text, label in zip(text_list, label_list):
        options = transform_f(text)
        aug_text_list.append(text)
        aug_text_list.extend(options)
        aug_label_list.append(label)
        aug_label_list.extend([label] * len(options))
    return aug_text_list, aug_label_list

from sklearn.cluster import KMeans
from scipy.sparse import vstack
from sklearn.cluster import KMeans
import numpy as np


# Not working fully
def cluster_sampling(X_train_bow, y_train):
    ''' Uses KMEANS and requires that X_train is alreayd in feature form e.g. Bag Of words or TF-IDF, vectorized form'''
    # Step 1: Cluster-based Sampling
    num_clusters = 5  # Number of clusters
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    clusters = kmeans.fit_predict(X_train_bow)

    # Sample size per cluster
    sample_size_per_cluster = 100

    X_train_sampled = []
    y_train_sampled = []

    for cluster_id in range(num_clusters):
        cluster_indices = (clusters == cluster_id).nonzero()[0]
        sampled_indices = np.random.choice(cluster_indices, size=min(sample_size_per_cluster, len(cluster_indices)), replace=False)
        X_train_sampled.extend(X_train_bow[sampled_indices])
        y_train_sampled.extend([y_train[i] for i in sampled_indices])
    return X_train_sampled, y_train_sampled

import random
import numpy as np
from googletrans import Translator
import os
import json
from deep_translator import GoogleTranslator

def translate_group(text_list, label_list):
    aug_text_list = []
    aug_label_list = []
    choices = ['fr', 'de', 'ar']
    translators_to = [GoogleTranslator(source='en', target=x) for x in choices]
    translators_from = [GoogleTranslator(source=x, target='en') for x in choices]
    for text, label in zip(text_list, label_list):
        aug_text_list.append(text)
        for i in range(3):
            new_text = [translators_from[i].translate(translators_to[i].translate(text))]
            aug_text_list.extend(new_text)
        aug_label_list.append(label)
        aug_label_list.extend([label] * len(new_text))
    return aug_text_list, aug_label_list

import pickle
def write_list_to_file(list1, filename1):
    with open(filename1, 'wb') as file:
        pickle.dump(list1, file)
def read_list_from_file(filename1):
    with open(filename1, 'rb') as file:
        return pickle.load(file)

res1, res2 = translate_group(train_data_text_list[:1] , train_data_label_list[:1])
for x in res1:
    print(x)

# write_list_to_file(train_data_text_list, "full_data_text_list.pkl")
# write_list_to_file(train_data_label_list, "full_data_label_list.pkl")

Augmented Text 1: somebody destruct my only play
Augmented Text 2: person put_down my only fiddle
Augmented Text 3: soul destruct my only flirt


KeyboardInterrupt: 

In [ ]:
# Computationally expensive transformations
X_aug, y_aug = replacement_text(train_data_text_list, train_data_label_list, synonym_replacement)

In [ ]:
# X,y = train_data_text_list, train_data_label_list
X,y = read_list_from_file("french_translation_text.txt"), read_list_from_file("french_translation_label.txt")

## Benchmark - Bag of words (no augmentation)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, f1_score
import numpy as np
from scipy.sparse import vstack
# Example dataset (X: text data, y: sentiment labels)
print(len(X))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define stopwords and create CountVectorizer with n-grams and stopwords removal
stop_words = list(stop_words)
vectorizer = CountVectorizer(ngram_range=(1, 2), stop_words=stop_words)

# Create BoW vectors
maxv = -float('inf')
X_train_bow = vectorizer.fit_transform(X_train)
X_test_bow = vectorizer.transform(X_test)

# Step 2: Train a logistic regression classifier on the sampled data
classifier = LogisticRegression()
classifier.fit(X_train_bow, y_train)

# Step 3: Evaluate the model
y_pred = classifier.predict(X_test_bow)
calc_scores(y_test, y_pred)

# Unseen data
unseen_data = test_data_text_list
true_labels = test_data_label_list

# Vectorize the unseen test data using the same CountVectorizer instance
X_unseen_bow = vectorizer.transform(unseen_data)

# Predict sentiment labels for the unseen test data
y_unseen_pred = classifier.predict(X_unseen_bow)

# Calculate f1_score
acc, f1 = calc_scores(true_labels, y_unseen_pred)

# calc_scores_and_print_miscalculations(true_labels, y_unseen_pred, unseen_data)
print(len(X_aug))

1200
Accuracy: 0.95
F1 Score: 0.9508196721311475
Accuracy: 0.7120343839541547
F1 Score: 0.31242873432155077
9528


## Benchmark model - SVM - TF-IDF feature representation

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

In [ ]:

# Split the dataset into training and testing sets
# X,y = train_data_text_list[:300] + train_data_text_list[-300:], train_data_label_list[:300] +  train_data_label_list[-300:]
X, y = train_data_text_list, train_data_label_list
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create TF-IDF vectors
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Train an SVM classifier
svm_classifier = SVC(kernel='linear')
svm_classifier.fit(X_train_tfidf, y_train)

# Predict sentiment labels for the test set
y_pred = svm_classifier.predict(X_test_tfidf)

calc_scores(y_test, y_pred)

# Example of unseen test data
unseen_data = test_data_text_list
true_labels = test_data_label_list # True sentiment labels for the unseen test data

# Convert the unseen test data into TF-IDF vectors
X_unseen_tfidf = tfidf_vectorizer.transform(unseen_data)

# Predict sentiment labels for the unseen test data
y_unseen_pred = svm_classifier.predict(X_unseen_tfidf)

# Calculate accuracy for unseen test data
calc_scores_and_print_miscalculations(true_labels, y_unseen_pred, unseen_data)


Accuracy: 0.9166666666666666
F1 Score: 0.9173553719008265
Accuracy: 0.6575931232091691
F1 Score: 0.2865671641791045

Misclassifications:
Pred: 1 Ac: 0 Text: The Scandinavian country is known for its cradle-to-grave welfare system , which includes affordable health care and generous social benefits for those in need .
Pred: 1 Ac: 0 Text: "The new immigrants also hit up the vendors because they "" did n't deliver what they were paid for "" ."
Pred: 1 Ac: 0 Text: An attacker within nearby range of a vulnerable device could trick the app into accepting their certificate , allowing them to siphon off any data to and from the app .
Pred: 1 Ac: 0 Text: "She added : "" I am really happy with the awards , but I just want to help those in need . """
Pred: 1 Ac: 0 Text: Part of the church 's collection also goes towards a special care fund which assists families in need to finance medical bills or school fees . The pastor said that though the fund is limited , countless persons have benefited fro

## Analysis - visualisation (matplotlib)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Combine the sequences, true labels, and predicted labels into a list of tuples
data = list(zip(unseen_data, true_labels, y_unseen_pred))

# Sort the data based on the length of the input sequences
sorted_data = sorted(data, key=lambda x: len(x[0].split()))

# Define the number of groups
n_groups = 5

# Calculate the group size
group_size = len(sorted_data) // (n_groups + 1)

# Divide the sorted data into n_groups approximately equal-sized groups
groups = [sorted_data[i:i+group_size] for i in range(0, len(sorted_data), group_size)]
print(len(groups))
# Initialize counters for correctly predicted labels
group_correct = [0] * (n_groups + 1)

# Iterate through each group and count the number of correctly predicted labels
for i, group in enumerate(groups):
    for seq, true_label, pred_label in group:
        if true_label == pred_label:
            group_correct[min(i, len(group_correct) - 1)] += 1  # Ensure index doesn't exceed the length of group_correct

# Calculate accuracy percentage for each group
group_sizes = [len(group) for group in groups]
group_accuracies = [(correct / size) * 100 if size != 0 else 0 for correct, size in zip(group_correct, group_sizes)]

# Plot the results
# Calculate the amount of characters at the end of each group
end_char_counts = [len(group[-1][0]) for group in groups]
categories = [f'{end_char_count} chars' for end_char_count in end_char_counts]
print(categories)
accuracies = group_accuracies

plt.bar(categories, accuracies, color=['skyblue'])
plt.xlabel('Input Sequence Length Group')
plt.ylabel('Accuracy Percentage')
plt.title('Accuracy Percentage by Input Sequence Length Group')
plt.ylim(0, 100)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.metrics import f1_score, accuracy_score

# Group data based on categories
data_categories = test_community  # True labels
pcl_strength = test_pcl
c2 = Counter(pcl_strength)
for k,v in c2.items():
    print(k,v)
c = Counter(data_categories)
for k,v in c.items():
    print(k, v)

category_data = {}
for category, true_label, pred_label, pcl_strength_val in zip(data_categories, true_labels, y_unseen_pred, pcl_strength):
    if category not in category_data:
        category_data[category] = {'true_labels': [], 'pred_labels': [], 'pcl_strength': []}
    category_data[category]['true_labels'].append(true_label)
    category_data[category]['pred_labels'].append(pred_label)
    category_data[category]['pcl_strength'].append(pcl_strength_val)

# Calculate F1 score for each category
category_f1_scores = {}
for category, data in category_data.items():
    category_f1_scores[category] = f1_score(data['true_labels'], data['pred_labels'])
    # category_f1_scores[category] = accuracy_score(data['true_labels'], data['pred_labels'])

# Plot the results
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(10, 12))

# Plot for data_categories
axes[0].bar(range(len(category_f1_scores)), category_f1_scores.values(), color='skyblue')
axes[0].set_xlabel('Data Categories')
axes[0].set_ylabel('F1 Score')
axes[0].set_title('F1 Score by Data Categories')
axes[0].set_xticks(range(len(category_f1_scores)))
axes[0].set_xticklabels(list(category_f1_scores.keys()), rotation=45, ha='right')
axes[0].grid(axis='y', linestyle='--', alpha=0.7)

# Plot for pcl_strength
pcl_strength_data = {}
for category, data in category_data.items():
    for pcl_strength_val, f1_score_val in zip(data['pcl_strength'], category_f1_scores.values()):
        if pcl_strength_val not in pcl_strength_data:
            pcl_strength_data[pcl_strength_val] = []
        pcl_strength_data[pcl_strength_val].append(f1_score_val)

# Sort pcl_strength_data by keys (PCL strength)
sorted_pcl_strength_data = dict(sorted(pcl_strength_data.items()))

axes[1].bar(range(len(sorted_pcl_strength_data)), [sum(scores)/len(scores) for scores in sorted_pcl_strength_data.values()], color='skyblue')
axes[1].set_xlabel('PCL Strength')
axes[1].set_ylabel('Average F1 Score')
axes[1].set_title('Average F1 Score by PCL Strength')
axes[1].set_xticks(range(len(sorted_pcl_strength_data)))
axes[1].set_xticklabels(sorted_pcl_strength_data.keys())
axes[1].grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()


# DeBERTa model

In [ ]:
!pip install simpletransformers
!pip install tensorboardx

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs

In [ ]:
# Prepare dataset for training and validation
x = train_data_text_list
y = train_data_label_list
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)

# Combining dataset for training
train_df = pd.DataFrame({'text':x_train, 'label':y_train})

# Defining testing variables
x_test = test_data_text_list
y_test_label = test_data_label_list

In [ ]:
def train_and_output_deberta(model_args, optimisation_name=""):
    # Define model

    deberta_model = ClassificationModel("deberta",
                                      'microsoft/deberta-base',
                                      args = model_args,
                                      num_labels=2,
                                      use_cuda=torch.cuda.is_available())

    # Training the model
    deberta_model.train_model(train_df)

    # Evaluate the model on the validation set
    y_pred, _ = deberta_model.predict(x_val)
    print("DeBERTa " + optimisation_name + ". Validation set")
    calc_scores(y_val, y_pred)

    # Evaluate the model on the test set
    y_test_pred, _ = deberta_model.predict(x_test)
    print("DeBERTa " + optimisation_name + ". Test set")
    calc_scores(y_test_label, y_test_pred)

## DeBERTa base model without augmentations


In [ ]:
deberta_base_model_args = ClassificationArgs(num_train_epochs=1,
                                      no_save=True,
                                      no_cache=True,
                                      overwrite_output_dir=True,
                                      use_multiprocessing=False,
                                      use_multiprocessing_for_evaluation=False,
                                      dataloader_num_workers=8,)

train_and_output_deberta(deberta_base_model_args, "Base model")

Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:610: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 1 of 2:   0%|          | 0/239 [00:00<?, ?it/s]

Running Epoch 2 of 2:   0%|          | 0/239 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

DeBERTa Base model. Validation set
Accuracy: 0.8301886792452831
F1 Score: 0.7395498392282959


  0%|          | 0/21 [00:00<?, ?it/s]

DeBERTa Base model. Test set
Accuracy: 0.8825214899713467
F1 Score: 0.5269230769230769


## DeBERTa optimisations

In [ ]:
# Define model args
deberta_epochs_model_args = ClassificationArgs(num_train_epochs=10,
                                      no_save=True,
                                      no_cache=True,
                                      overwrite_output_dir=True,
                                      use_multiprocessing=False,
                                      use_multiprocessing_for_evaluation=False,
                                      dataloader_num_workers=8,)

train_and_output_deberta(deberta_epochs_model_args, "10 epochs")

Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:610: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 1 of 10:   0%|          | 0/239 [00:00<?, ?it/s]

Running Epoch 2 of 10:   0%|          | 0/239 [00:00<?, ?it/s]

Running Epoch 3 of 10:   0%|          | 0/239 [00:00<?, ?it/s]

...

Running Epoch 10 of 10:   0%|          | 0/239 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

DeBERTa 10 epochs. Validation set
Accuracy: 0.8050314465408805
F1 Score: 0.7155963302752293


  0%|          | 0/21 [00:00<?, ?it/s]

DeBERTa 10 epochs. Test set
Accuracy: 0.8576886341929322
F1 Score: 0.5033333333333334


In [ ]:
# Define model args
deberta_epochs_model_args = ClassificationArgs(num_train_epochs=5,
                                      no_save=True,
                                      no_cache=True,
                                      overwrite_output_dir=True,
                                      use_multiprocessing=False,
                                      use_multiprocessing_for_evaluation=False,
                                      dataloader_num_workers=8,)

train_and_output_deberta(deberta_epochs_model_args, "5 epochs")

Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:610: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 1 of 5:   0%|          | 0/239 [00:00<?, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/239 [00:00<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/239 [00:00<?, ?it/s]

...

Running Epoch 5 of 5:   0%|          | 0/239 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

DeBERTa 5 epochs. Validation set
Accuracy: 0.8134171907756813
F1 Score: 0.7374631268436578


  0%|          | 0/21 [00:00<?, ?it/s]

DeBERTa 5 epochs. Test set
Accuracy: 0.839541547277937
F1 Score: 0.47987616099071206


In [ ]:
# Define model args
deberta_epochs_model_args = ClassificationArgs(num_train_epochs=3,
                                      no_save=True,
                                      no_cache=True,
                                      overwrite_output_dir=True,
                                      use_multiprocessing=False,
                                      use_multiprocessing_for_evaluation=False,
                                      dataloader_num_workers=8,)

train_and_output_deberta(deberta_epochs_model_args, "3 epochs")

Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:610: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/239 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/239 [00:00<?, ?it/s]

Running Epoch 3 of 3:   0%|          | 0/239 [00:01<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

DeBERTa 3 epochs. Validation set
Accuracy: 0.7882599580712788
F1 Score: 0.7154929577464788


  0%|          | 0/21 [00:00<?, ?it/s]

DeBERTa 3 epochs. Test set
Accuracy: 0.833810888252149
F1 Score: 0.48367952522255186


In [ ]:
# Define model args
deberta_epochs_model_args = ClassificationArgs(num_train_epochs=1,
                                      learning_rate=1e-5,
                                      no_save=True,
                                      no_cache=True,
                                      overwrite_output_dir=True,
                                      use_multiprocessing=False,
                                      use_multiprocessing_for_evaluation=False,
                                      dataloader_num_workers=8,)

train_and_output_deberta(deberta_epochs_model_args, "Learning rate 1e-5")

Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:610: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 1 of 1:   0%|          | 0/239 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

DeBERTa Learnign rate 1e-5. Validation set
Accuracy: 0.7924528301886793
F1 Score: 0.6990881458966566


  0%|          | 0/21 [00:00<?, ?it/s]

DeBERTa Learnign rate 1e-5. Test set
Accuracy: 0.8519579751671442
F1 Score: 0.47986577181208045


In [ ]:
# Define model args
deberta_epochs_model_args = ClassificationArgs(num_train_epochs=5,
                                      learning_rate=1e-5,
                                      no_save=True,
                                      no_cache=True,
                                      overwrite_output_dir=True,
                                      use_multiprocessing=False,
                                      use_multiprocessing_for_evaluation=False,
                                      dataloader_num_workers=8,)

train_and_output_deberta(deberta_epochs_model_args, "Learning rate 1e-5, 5 epochs")

Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:610: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 1 of 5:   0%|          | 0/239 [00:02<?, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/239 [00:03<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/239 [00:00<?, ?it/s]

...

Running Epoch 5 of 5:   0%|          | 0/239 [00:02<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

DeBERTa Learning rate 1e-5, 5 epochs. Validation set
Accuracy: 0.8427672955974843
F1 Score: 0.7787610619469026


  0%|          | 0/21 [00:00<?, ?it/s]

DeBERTa Learning rate 1e-5, 5 epochs. Test set
Accuracy: 0.876313276026743
F1 Score: 0.5542168674698795


In [ ]:
# Define model args
deberta_epochs_model_args = ClassificationArgs(num_train_epochs=1,
                                      learning_rate=1e-4,
                                      no_save=True,
                                      no_cache=True,
                                      overwrite_output_dir=True,
                                      use_multiprocessing=False,
                                      use_multiprocessing_for_evaluation=False,
                                      dataloader_num_workers=8,)

train_and_output_deberta(deberta_epochs_model_args, "Learning rate 1e-4") # Too big

Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:610: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 1 of 1:   0%|          | 0/239 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

DeBERTa Learning rate 1e-4. Validation set
Accuracy: 0.7442348008385744
F1 Score: 0.5763888888888888


  0%|          | 0/21 [00:00<?, ?it/s]

DeBERTa Learning rate 1e-4. Test set
Accuracy: 0.8319006685768864
F1 Score: 0.37142857142857144


In [ ]:
# Define model args
deberta_epochs_model_args = ClassificationArgs(num_train_epochs=10,
                                      learning_rate=1e-5,
                                      no_save=True,
                                      no_cache=True,
                                      overwrite_output_dir=True,
                                      use_multiprocessing=False,
                                      use_multiprocessing_for_evaluation=False,
                                      dataloader_num_workers=8,)

train_and_output_deberta(deberta_epochs_model_args, "Learning rate 1e-5, 10 epochs")

Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:610: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 1 of 10:   0%|          | 0/239 [00:00<?, ?it/s]

Running Epoch 2 of 10:   0%|          | 0/239 [00:03<?, ?it/s]

Running Epoch 3 of 10:   0%|          | 0/239 [00:01<?, ?it/s]

...

Running Epoch 10 of 10:   0%|          | 0/239 [00:03<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

DeBERTa Learning rate 1e-5, 10 epochs. Validation set
Accuracy: 0.8155136268343816
F1 Score: 0.7411764705882354


  0%|          | 0/21 [00:00<?, ?it/s]

DeBERTa Learning rate 1e-5, 10 epochs. Test set
Accuracy: 0.8720152817574021
F1 Score: 0.541095890410959


In [ ]:
# Define model args
deberta_epochs_model_args = ClassificationArgs(num_train_epochs=5,
                                      learning_rate=1e-5,
                                      weight_decay=1e-6,
                                      no_save=True,
                                      no_cache=True,
                                      overwrite_output_dir=True,
                                      use_multiprocessing=False,
                                      use_multiprocessing_for_evaluation=False,
                                      dataloader_num_workers=8,)

train_and_output_deberta(deberta_epochs_model_args, "Learning rate 1e-5, weight decay 1e-6, 5 epochs")

Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:610: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 1 of 5:   0%|          | 0/239 [00:00<?, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/239 [00:02<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/239 [00:03<?, ?it/s]

...

Running Epoch 5 of 5:   0%|          | 0/239 [00:01<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

DeBERTa Learning rate 1e-5, weight decay 1e-6, 5 epochs. Validation set
Accuracy: 0.8301886792452831
F1 Score: 0.7652173913043478


  0%|          | 0/21 [00:00<?, ?it/s]

DeBERTa Learning rate 1e-5, weight decay 1e-6, 5 epochs. Test set
Accuracy: 0.8701050620821394
F1 Score: 0.5374149659863946


In [ ]:
# Define model args
deberta_epochs_model_args = ClassificationArgs(num_train_epochs=10,
                                      learning_rate=1e-5,
                                      weight_decay=1e-6,
                                      no_save=True,
                                      no_cache=True,
                                      overwrite_output_dir=True,
                                      use_multiprocessing=False,
                                      use_multiprocessing_for_evaluation=False,
                                      dataloader_num_workers=8,)

train_and_output_deberta(deberta_epochs_model_args, "Learning rate 1e-5, weight decay 1e-6, 10 epochs")

Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:610: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 1 of 10:   0%|          | 0/239 [00:01<?, ?it/s]

Running Epoch 2 of 10:   0%|          | 0/239 [00:03<?, ?it/s]

Running Epoch 3 of 10:   0%|          | 0/239 [00:01<?, ?it/s]

...

Running Epoch 10 of 10:   0%|          | 0/239 [00:01<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

DeBERTa Learning rate 1e-5, weight decay 1e-6, 10 epochs. Validation set
Accuracy: 0.8343815513626834
F1 Score: 0.7538940809968847


  0%|          | 0/21 [00:00<?, ?it/s]

DeBERTa Learning rate 1e-5, weight decay 1e-6, 10 epochs. Test set
Accuracy: 0.8863419293218721
F1 Score: 0.5592592592592592


In [ ]:
# Define model args
deberta_epochs_model_args = ClassificationArgs(num_train_epochs=10,
                                      learning_rate=1e-5,
                                      weight_decay=1e-5,
                                      no_save=True,
                                      no_cache=True,
                                      overwrite_output_dir=True,
                                      use_multiprocessing=False,
                                      use_multiprocessing_for_evaluation=False,
                                      dataloader_num_workers=8,)

train_and_output_deberta(deberta_epochs_model_args, "Learning rate 1e-5, weight decay 1e-5, 10 epochs")

Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:610: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 1 of 10:   0%|          | 0/239 [00:00<?, ?it/s]

Running Epoch 2 of 10:   0%|          | 0/239 [00:01<?, ?it/s]

Running Epoch 3 of 10:   0%|          | 0/239 [00:00<?, ?it/s]

...

Running Epoch 10 of 10:   0%|          | 0/239 [00:03<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

DeBERTa Learning rate 1e-5, weight decay 1e-5, 10 epochs. Validation set
Accuracy: 0.8134171907756813
F1 Score: 0.7359050445103857


  0%|          | 0/21 [00:00<?, ?it/s]

DeBERTa Learning rate 1e-5, weight decay 1e-5, 10 epochs. Test set
Accuracy: 0.8638968481375359
F1 Score: 0.5226130653266331


In [ ]:
# Define model args
deberta_epochs_model_args = ClassificationArgs(num_train_epochs=10,
                                      learning_rate=1e-5,
                                      scheduler="linear_schedule_with_warmup",
                                      warmup_steps=100,
                                      warmup_ratio=0.1,
                                      use_early_stopping=True,
                                      early_stopping_patience=3,
                                      early_stopping_delta=0.01,
                                      no_save=True,
                                      no_cache=True,
                                      overwrite_output_dir=True,
                                      use_multiprocessing=False,
                                      use_multiprocessing_for_evaluation=False,
                                      dataloader_num_workers=8,)

# I don't know if early stopping worked here or not. Because it still run for 10 epochs
train_and_output_deberta(deberta_epochs_model_args, "Learning rate 1e-5, 10 epochs, early stopping, learning scheduler")

Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:610: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 1 of 10:   0%|          | 0/239 [00:01<?, ?it/s]

Running Epoch 2 of 10:   0%|          | 0/239 [00:03<?, ?it/s]

Running Epoch 3 of 10:   0%|          | 0/239 [00:01<?, ?it/s]

...

Running Epoch 10 of 10:   0%|          | 0/239 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

DeBERTa Learning rate 1e-5, 10 epochs, early stopping, learning scheduler. Validation set
Accuracy: 0.8071278825995807
F1 Score: 0.7325581395348837


  0%|          | 0/21 [00:00<?, ?it/s]

DeBERTa Learning rate 1e-5, 10 epochs, early stopping, learning scheduler. Test set
Accuracy: 0.8672397325692455
F1 Score: 0.5366666666666666


In [ ]:
deberta_base_model_args = ClassificationArgs(
    num_train_epochs=3,
    learning_rate=1e-5,
    scheduler="linear_schedule_with_warmup",
    use_early_stopping=True,
    early_stopping_patience=3,
    early_stopping_delta=0.01,
    # Specify additional scheduler parameters if needed
    warmup_steps=100,  # Number of steps for the warmup phase
    warmup_ratio=0.1,  # Or specify ratio instead of fixed steps
    # Other arguments
    no_save=True,
    no_cache=True,
    overwrite_output_dir=True
)
weight_decay